In [1]:
from django import setup
setup()

In [2]:
from django.utils import timezone
from entries.models import Entry, Nutrient, UserWeightHistory

In [3]:
e1 = Entry(when=timezone.now(), what="something 4", quantity=2, category="c", measure="g")

In [4]:
e1.save()
e1.id

126

In [5]:
i1 = Nutrient()

In [6]:
i1.label = "sugar"
i1.unit = "g"
i1.category = "i"
i1.quantity = 4

In [7]:
from django.db.models import Sum, Count
Nutrient.objects.filter(category="o").aggregate(Sum("quantity"))

{'quantity__sum': 1.32}

## Reporting

In [3]:
from django.db.models.functions import TruncDay, TruncDate
from django.db.models import Sum, Count

In [4]:
import arrow
start_date = arrow.get("2016-11-16")
end_date = arrow.get("2016-12-06 23:59:59")
print(start_date, end_date)

(<Arrow [2016-11-16T00:00:00+00:00]>, <Arrow [2016-12-06T23:59:59+00:00]>)


In [ ]:
e1 = Entry(
    when=timezone.now(), 
    what="something 4", 
    quantity=2, 
    category=Entry.CATEGORIES.FOOD_CONSUMPTION, 
    measure="g")
e1.save()

e2 = Entry(
    when=timezone.now(), 
    what="something 4", 
    quantity=2, 
    category=Entry.CATEGORIES.PHYSICAL_ACTIVITY, 
    measure="g")

e2.save()

In [ ]:
e1n1 = Nutrient.objects.create(
    entry=e1,
    category=Nutrient.CATEGORIES.INTAKE,
    label="Energy",
    unit="kcal",
    quantity=140.5
)

e2n1 = Nutrient.objects.create(
    entry=e1,
    category=Nutrient.CATEGORIES.OUTTAKE,
    label="Energy",
    unit="kcal",
    quantity=10.5
)



In [6]:
Nutrient.get_energy_report(
    Nutrient.CATEGORIES.INTAKE, 
    timezone.make_aware(start_date.naive), 
    timezone.make_aware(arrow.utcnow().naive))

{datetime.date(2016, 11, 16): 0,
 datetime.date(2016, 11, 17): 0,
 datetime.date(2016, 11, 18): 0,
 datetime.date(2016, 11, 19): 0,
 datetime.date(2016, 11, 20): 0,
 datetime.date(2016, 11, 21): 0,
 datetime.date(2016, 11, 22): 0,
 datetime.date(2016, 11, 23): 0,
 datetime.date(2016, 11, 24): 0,
 datetime.date(2016, 11, 25): 0,
 datetime.date(2016, 11, 26): 0,
 datetime.date(2016, 11, 27): 0,
 datetime.date(2016, 11, 28): 0,
 datetime.date(2016, 11, 29): 0,
 datetime.date(2016, 11, 30): 90.0,
 datetime.date(2016, 12, 1): 28.0,
 datetime.date(2016, 12, 2): 0,
 datetime.date(2016, 12, 3): 0,
 datetime.date(2016, 12, 4): 0,
 datetime.date(2016, 12, 5): 0,
 datetime.date(2016, 12, 6): 0,
 datetime.date(2016, 12, 7): 0}

In [59]:
report = Nutrient.get_energy_report(
    Nutrient.CATEGORIES.OUTTAKE, 
    timezone.make_aware(start_date.naive), 
    timezone.make_aware(end_date.naive))

sorted(report.items())

[(datetime.date(2016, 11, 16), 0),
 (datetime.date(2016, 11, 17), 0),
 (datetime.date(2016, 11, 18), 0),
 (datetime.date(2016, 11, 19), 0),
 (datetime.date(2016, 11, 20), 0),
 (datetime.date(2016, 11, 21), 0),
 (datetime.date(2016, 11, 22), 0),
 (datetime.date(2016, 11, 23), 0),
 (datetime.date(2016, 11, 24), 0),
 (datetime.date(2016, 11, 25), 0),
 (datetime.date(2016, 11, 26), 0),
 (datetime.date(2016, 11, 27), 0),
 (datetime.date(2016, 11, 28), 0),
 (datetime.date(2016, 11, 29), 0),
 (datetime.date(2016, 11, 30), 0),
 (datetime.date(2016, 12, 1), 0),
 (datetime.date(2016, 12, 2), 1.32),
 (datetime.date(2016, 12, 3), 0),
 (datetime.date(2016, 12, 4), 0),
 (datetime.date(2016, 12, 5), 0),
 (datetime.date(2016, 12, 6), 12.5)]

In [38]:
a = (end_date - start_date)

In [52]:
records = Nutrient.objects.values("label", "unit").annotate(value=Sum("quantity"))
print(records.query.__str__())
for i in records:
    print(i)


SELECT "entries_nutrient"."label", "entries_nutrient"."unit", SUM("entries_nutrient"."quantity") AS "value" FROM "entries_nutrient" GROUP BY "entries_nutrient"."label", "entries_nutrient"."unit"
{'value': 13.82, 'unit': u'kcal', 'label': u'Energy'}
{'value': 8.0, 'unit': u'mg', 'label': u'fat'}


In [24]:
e2n1 = Nutrient.objects.create(
    entry=e1,
    category=Nutrient.CATEGORIES.OUTTAKE,
    label="fat",
    unit="mg",
    quantity=3
)


In [43]:
nutrients = Nutrient.objects.filter(
        category="o",
        entry__when__range=[timezone.make_aware(start_date.naive), 
            timezone.make_aware(end_date.naive)]
    ).values(
        "label", "unit"
    ).annotate(
        value=Sum("quantity")
    )

In [42]:
nutrients.query.__str__()

u'SELECT "entries_nutrient"."label", "entries_nutrient"."unit", SUM("entries_nutrient"."quantity") AS "value" FROM "entries_nutrient" INNER JOIN "entries_entry" ON ("entries_nutrient"."entry_id" = "entries_entry"."id") WHERE ("entries_nutrient"."category" = o AND "entries_entry"."when" BETWEEN 2016-11-16 00:00:00 AND 2016-12-06 23:59:59) GROUP BY "entries_nutrient"."label", "entries_nutrient"."unit"'

In [ ]:

    @classmethod
    def get_nutrients_report(cls, start_date, end_date):
        nutrients = Nutrient.objects.filter(
            category=cls.CATEGORIES.INTAKE,
            entry__when__range=[start_date, end_date]
        ).values(
            "label", "unit"
        ).annotate(
            value=Sum("quantity")
        )
        return nutrients

In [67]:
def create_time_series(start_date, end_date):
    """
    creates a time series sequence taken at successive daily spaced points
    between start_date end end_date

    Args:
        start_date:
        end_date:

    Returns:

    """
    current_date = arrow.get(start_date)
    end_date = arrow.get(end_date)
    data = {}

    while current_date <= end_date:
        data[current_date.datetime] = 0
        current_date = current_date.replace(days=1)

    return data
create_time_series(start_date, end_date)

{datetime.datetime(2016, 11, 16, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 17, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 18, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 19, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 20, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 21, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 22, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 23, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 24, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 25, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 26, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 27, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 28, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 29, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 11, 30, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 12, 1, 0, 0, tzinfo=tzutc()): 0,
 datetime.datetime(2016, 12, 2, 0, 0, tzinfo=tzutc()): 0,

In [77]:
data = create_time_series(start_date, end_date)

records = Nutrient.objects.filter(
    label="Energy",
    unit="kcal",
    category="o",
    entry__when__range=[timezone.make_aware(start_date.naive), timezone.make_aware(end_date.naive)]
).annotate(
    date=TruncDate("entry__when"),
).values(
    "date"
).annotate(
    value=Sum('quantity')
).values(
    "date", "value", "entry__when"
)

for i in records:
    data[i["date"]] = i["value"]
    print(i["entry__when"])

2016-12-02 15:23:03+00:00
2016-12-06 15:24:36.147835+00:00


In [81]:
timezone.make_aware(start_date.datetime)

ValueError: Not naive datetime (tzinfo is already set)

In [83]:
start_date.datetime

datetime.datetime(2016, 11, 16, 0, 0, tzinfo=tzutc())

In [3]:
UserWeightHistory.objects.create(weight=71)

OperationalError: no such table: entries_userweighthistory